In [1]:
# !pip install lxml
# !pip install pandas
import pandas as pd
import lxml
import xml.etree.ElementTree as ET
f_ratios = pd.read_json("financial_ratios.jsonl", lines=True)
f_ratios = f_ratios.rename(columns={"cust_num": "customer_id"}).astype({"customer_id": int})

f_ratios["monthly_income"] = (
    f_ratios["monthly_income"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)

f_ratios["existing_monthly_debt"] = (
    f_ratios["existing_monthly_debt"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)

f_ratios["monthly_payment"] = (
    f_ratios["monthly_payment"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)
f_ratios["debt_to_income_ratio"] = (f_ratios["debt_to_income_ratio"].round(2))
f_ratios["debt_service_ratio"] = (f_ratios["debt_service_ratio"].round(2))
f_ratios["payment_to_income_ratio"] = (f_ratios["payment_to_income_ratio"].round(2))
f_ratios["credit_utilization"] = (f_ratios["credit_utilization"].round(2))
f_ratios["loan_to_annual_income"] = (f_ratios["loan_to_annual_income"].round(2))
f_ratios["revolving_balance"] = f_ratios["revolving_balance"].fillna(0)

f_ratios["revolving_balance"] = (
    f_ratios["revolving_balance"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)

f_ratios["credit_usage_amount"] = (
    f_ratios["credit_usage_amount"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)

f_ratios["available_credit"] = (
    f_ratios["available_credit"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)

f_ratios["total_monthly_debt_payment"] = (
    f_ratios["total_monthly_debt_payment"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)
f_ratios["total_debt_amount"] = (
    f_ratios["total_debt_amount"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)
f_ratios["monthly_free_cash_flow"] = (
    f_ratios["monthly_free_cash_flow"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)
f_ratios["annual_debt_payment"] = (
    f_ratios["annual_debt_payment"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)
f_ratios.to_csv("data1/cleaned_financial_ratios.csv", index=False)

df = pd.read_parquet('credit_history.parquet')


if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

if "num_delinquencies_2yrs" in df.columns:
    df['num_delinquencies_2yrs'] = df['num_delinquencies_2yrs'].fillna(0)
else:
    print("Column 'num_delinquencies_2yrs' not found!")

if "customer_number" in df.columns:
    df = df.rename(columns={"customer_number": "customer_id"})
else:
    print("Column 'customer_number' not found!")

df['oldest_account_age_months'] = (
    df['oldest_account_age_months']
        .astype(str)
        .str.replace("'", " ")
        .str.replace('"', " ")
        .str.strip()
)

df['oldest_account_age_months'] = pd.to_numeric(df['oldest_account_age_months'], errors='coerce').fillna(0)
df['oldest_account_age_months'].dtype

df.to_csv("data1/cleaned_credit_history.csv", index=False)




df1 = pd.read_xml("geographic_data.xml")
unnamed_cols = [col for col in df1.columns if "unnamed" in col.lower()]
df1 = df1.drop(columns=unnamed_cols)

rename_map = {}
for col in df1.columns:
    if col.lower() == "id":
        rename_map[col] = "customer_id"
df1 = df1.rename(columns=rename_map)

for col in df1.columns:
    df1[col] = df1[col].map(lambda x: x.strip().replace("'", "") if isinstance(x, str) else x)


for col in df1.columns:
    df1[col] = pd.to_numeric(df1[col], errors="coerce").fillna(df1[col])

df1 = df1.fillna(0)
df1.to_csv("data1/cleaned_geographic_data.csv", index=False)

loan_details=pd.read_excel("loan_details.xlsx")

loan_type_clean_map = {
    'personal': 'Personal Loan',
    'personal loan': 'Personal Loan',
    'personal': 'Personal Loan',
    'personal loan': 'Personal Loan',

    'creditcard': 'Credit Card',
    'credit card': 'Credit Card',
    'cc': 'Credit Card',

    'mortgage': 'Mortgage',
    'home loan': 'Mortgage'
}

loan_details['loan_type_cleaned'] = (
    loan_details['loan_type']
    .str.strip()
    .str.lower()
    .map(loan_type_clean_map)
)

loan_details['loan_amount_clean'] = (
    loan_details['loan_amount']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

loan_details['interest_rate'] = loan_details['interest_rate'].astype(float) / 100

loan_details=loan_details.drop(columns={'loan_amount','loan_type'})


loan_details.rename(columns={'loan_type_cleaned':'loan_type','loan_amount_clean':'loan_amount'})

loan_mapping = {
    'Debt Consolidation': 0,
    'Refinance': 1,
    'Major Purchase': 2,
    'Medical': 3,
    'Revolving Credit': 4,
    'Home Improvement': 5,
    'Home Purchase': 6,
    'Other': 7
}

loan_details['loan_purpose_encoded'] = loan_details['loan_purpose'].map(loan_mapping)


channel_mapping = {
    'Direct Mail': 0,
    'Branch': 1,
    'Online': 2,
    'Broker': 3
}

loan_details['origination_channel_encoded'] = loan_details['origination_channel'].map(channel_mapping)
loan_type_mapping = {
    'Personal Loan': 0,
    'Mortgage': 1,
    'Credit Card': 2
}
loan_details['loan_type_encoded'] = loan_details['loan_type_cleaned'].map(loan_type_mapping)

columns_needed = [
    'customer_id', 
    'loan_term', 
    'interest_rate', 
    'loan_to_value_ratio', 
    'loan_officer_id', 
    'marketing_campaign', 
    'loan_amount_clean', 
    'loan_purpose_encoded', 
    'origination_channel_encoded', 
    'loan_type_encoded'
]

loan_final = loan_details[columns_needed].copy()

loan_final.rename(columns={
    'loan_amount_clean': 'loan_amount',
    'loan_purpose_encoded': 'loan_purpose',
    'origination_channel_encoded': 'origination_channel',
    'loan_type_encoded': 'loan_type'
}, inplace=True)

loan_final.to_csv('data1/cleaned_loan_details.csv', index=False)

application_metadata=pd.read_csv("application_metadata.csv")

contact_mapping = {
    'Mail': 0,
    'Email': 0,
    'Phone': 1
}

application_metadata['preferred_contact_encoded'] = application_metadata['preferred_contact'].map(contact_mapping)

account_status_mapping = {
    'ACT-1': 0,
    'ACT-2': 1,
    'ACT-3': 2,
    'A01': 3,
    'ACTIVE': 4
}

application_metadata['account_status_encoded'] = application_metadata['account_status_code'].map(account_status_mapping)

application_metadata=application_metadata.drop(columns={'preferred_contact','account_status_code'})
application_metadata=application_metadata.rename(columns={'customer_ref':'customer_id'})

application_metadata.to_csv('data1/cleaned_application_meta.csv', index=False)
demographics = pd.read_csv("demographics.csv")
demographics.head()
demographics.rename(columns={"cust_id": "customer_id"}, inplace=True)
demographics = demographics.astype({"customer_id": int,
                                   "age": int,
                                   "employment_length": float
                                   })

demographics["annual_income"] = (
    demographics["annual_income"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', '0')
    .astype(float).round(2)
)

demographics["employment_length"] = demographics["employment_length"].fillna(0)

demographics["employment_type"] = (
    demographics["employment_type"]
    .str.lower()
    .str.replace(r'[\s\-]+', '_', regex=True)
    .replace(
        {
            r'^(ft|ft:|fulltime|full_time)$': '0',
            r'^(pt|pt:|parttime|part_time)$': '1',
            r'^(self[_\- ]?emp:?|self_employed|self_employed:?)$': '2',
            "contractor": "3",
            "contract": "3",
            
        },
        regex=True,
    ).astype(int)
)
demographics["education"] = demographics["education"].str.lower().map(
        {
            "advanced": 0,
            "bachelor": 1,
            "graduate": 2,
            "high school": 3,
            "some college": 4,
        }
    ).astype(int)


demographics["marital_status"] = demographics["marital_status"].str.lower().map(
        {
            "single": 0,
            "married": 1,
            "divorced": 2,
            
        }
    ).astype(int)   
demographics.to_csv("data1/cleaned_demographics.csv", index=False)
demographics.isna().sum()
f_ratios
demographics
credit_history = pd.read_csv("data1/cleaned_credit_history.csv")
geographic_data = pd.read_csv("data1/cleaned_geographic_data.csv")
application_metadata = pd.read_csv("data1/cleaned_application_meta.csv")
loan_details = pd.read_csv("data1/cleaned_loan_details.csv")
merged = (
    application_metadata
    .merge(demographics, on="customer_id", how="left")
    .merge(credit_history, on="customer_id", how="left")
    .merge(geographic_data, on="customer_id", how="left")
    .merge(loan_details, on="customer_id", how="left")
    .merge(f_ratios, on="customer_id", how="left")
)

merged.to_csv("data/merged_final_data.csv", index=False)
merged

,customer_id,application_id,application_hour,application_day_of_week,account_open_year,referral_code,random_noise_1,num_login_sessions,num_customer_service_calls,has_mobile_app,...,payment_to_income_ratio,credit_utilization,revolving_balance,credit_usage_amount,available_credit,total_monthly_debt_payment,annual_debt_payment,loan_to_annual_income,total_debt_amount,monthly_free_cash_flow
0,100000,618540,21,0,2018,REF0000,1.141229,12,2,0,...,0.12,0.35,49772.4,49772.4,90827.6,1547.15,18565.80,0.22,61072.4,2811.18
1,100001,589784,9,6,2021,REF0000,0.943839,10,4,0,...,0.05,0.28,25492.8,25492.8,64907.2,1410.09,16921.08,0.15,32292.8,2398.24
2,100002,554219,10,4,2019,REF0000,-0.116033,4,3,1,...,0.06,0.38,12214.8,12214.8,20185.2,361.53,4338.36,0.17,15614.8,1305.14
3,100003,613505,3,4,2011,REF0000,-0.040562,7,0,1,...,0.10,0.36,14271.5,14271.5,24828.5,1477.20,17726.40,0.27,26271.5,2214.47
4,100004,571219,9,1,2021,REF0000,0.819017,6,0,1,...,0.54,0.49,23471.0,23471.0,24429.0,2572.34,30868.08,6.81,303371.0,852.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,109996,551059,11,6,2020,REF0000,-0.595108,7,2,0,...,0.59,0.64,16090.8,16090.8,9209.2,1841.21,22094.52,8.44,291090.8,875.46
9997,109997,599680,16,5,2018,REF0000,-0.037587,6,3,1,...,0.17,0.67,39094.2,39094.2,19605.8,791.54,9498.48,0.30,47294.2,1466.79
9998,109998,621614,3,0,2014,REF0000,0.454157,4,1,1,...,0.24,0.56,27417.0,27417.0,21983.0,867.83,10413.96,0.76,45117.0,1082.17
9999,109999,545654,12,5,2014,REF0000,0.682559,6,0,0,...,0.16,0.23,13506.5,13506.5,45993.5,681.22,8174.64,0.57,33406.5,2218.78


customer_id          0
age                  0
annual_income        0
employment_length    0
employment_type      0
education            0
marital_status       0
num_dependents       0
dtype: int64